In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from pandas.io.html import read_html
import re
import requests
import gurobipy as gp
from gurobipy import GRB
from math import *
from tqdm import tqdm

In [3]:
base_url = "https://www.investopedia.com/top-stocks-4581225"

content = requests.get(base_url)

urls_raw = re.findall("<li class=\"journey-nav__sublist-item \">\n.*\n.*\n", content.text) 

urls=[]

for url in urls_raw:
    url_clean = re.findall("https.*\"",url)
    url_cleaner = url_clean[0][:-1]
    urls.append(url_cleaner)

['https://www.investopedia.com/top-communications-stocks-4583180',
 'https://www.investopedia.com/investing/consumer-cyclical-stocks/',
 'https://www.investopedia.com/investing/consumer-defensive-stocks/',
 'https://www.investopedia.com/top-energy-stocks-4582081',
 'https://www.investopedia.com/top-financial-stocks-4582168',
 'https://www.investopedia.com/investing/top-healthcare-stocks/',
 'https://www.investopedia.com/top-industrial-stocks-4582171',
 'https://www.investopedia.com/top-materials-stocks-4582152',
 'https://www.investopedia.com/top-reits-4582128',
 'https://www.investopedia.com/top-tech-stocks-4581295',
 'https://www.investopedia.com/top-utilities-stocks-4582243',
 'https://www.investopedia.com/best-dividend-stocks-4774650',
 'https://www.investopedia.com/investing/best-growth-stocks/',
 'https://www.investopedia.com/investing/top-small-cap-stocks/',
 'https://www.investopedia.com/updates/top-penny-stocks/',
 'https://www.investopedia.com/updates/penny-stocks-buy-technic

In [4]:
urls_m = urls[0:10+1] + urls[18:]
tables = []
try:
    for url in urls_m:
        table = read_html(url)
        tables.append(table)
except Exception as e:
    print(url)
    
tables

[[  Best Value Communications Stocks Best Value Communications Stocks.1  \
  0                              NaN                          Price ($)   
  1            ViacomCBS Inc. (VIAC)                              28.82   
  2   Lumen Technologies Inc. (LUMN)                              10.05   
  3           Discovery Inc. (DISCK)                              19.12   
  
    Best Value Communications Stocks.2 Best Value Communications Stocks.3  
  0                    Market Cap ($B)        12-Month Trailing P/E Ratio  
  1                               17.8                                7.1  
  2                               11.0                                8.9  
  3                                9.7                                9.7  ,
    Fastest Growing Communications Stocks  \
  0                                   NaN   
  1   Zoom Video Communications Inc. (ZM)   
  2       Liberty Broadband Corp. (LBRDA)   
  3         John Wiley & Sons Inc. (JW.A)   
  
    Fastest G

In [5]:
# Check if very webpage follow the value, growth, momentum format.
for table in tables:
    if(len(table) != 3):
        print(len(table))

In [6]:
stocks_raw = []
tags = []

for table in tables:
    for i in range(3):
        stocks =  table[i].iloc[1:3+1,0]
        identifier = stocks.name
        identifier = identifier.split(" ")
        if("Value" in identifier): identifier = [e for e in identifier if e not in {"Best","Stocks"}]
        if("Growing" in identifier) : identifier = [e for e in identifier if e not in {"Fastest","Stocks"}]
        if("Momentum" in identifier) : identifier = [e for e in identifier if e not in {"with","the","Most","Stocks"}]

        stocks_raw.append(stocks)
        tags.append(identifier)

In [7]:
stocks_cleaned = []
for i in range(len(stocks_raw)):
    for j in range(len(stocks_raw[i])):
        stocks_df = list(stocks_raw[i])[j]
        stocks_cleaned.append([stocks_df,tags[i]])

In [7]:
stocks_rows = []

for i in range(len(stocks_cleaned)):
    stock = stocks_cleaned[i][0]
    tag = stocks_cleaned[i][1].copy()
    
    stock_raw = stock.split(" ")
    stock_name = " ".join(stock_raw[:-1])
    stock_ticker = stock_raw[-1][1:-1]
    # type: value,growing or momentum
    if ("Value" in tag):
        type_ = "Value"
        tag.remove("Value")
    if ("Growing" in tag):
        type_ = "Growth"
        tag.remove("Growing")
    if ("Momentum" in tag):
        type_ = "Momentum"
        tag.remove("Momentum")
    
    # Sub-categories
    sub = " ".join(tag)
    stocks_rows.append([stock_name,stock_ticker,type_,sub])
    

In [8]:
def expand_rows(lst):
    types = ['Growth', 'Momentum', 'Value']
    a = lst[0:2]
    b = [int(lst[2] in type) for type in types]
    c = [lst[3]]
    return(a+b+c)

In [24]:
stocks_expanded = list(map(lambda x:expand_rows(x),stocks_rows))

df_stocks = pd.DataFrame(stocks_expanded,columns=["stock","ticker",'is.Growth', 'is.Momentum', 'is.Value',
                                                  "Tag"])

df_stocks.loc[df_stocks.ticker == "JW.A", 'ticker'] = "JW-A"
def convert_outliers(x):
    words = x.split(" ")
    outlier_words = ["Beer", "Coal","Coffee","Wind","Copper","Cosmetics","Energy","Railway","Steel"]
    if(len(words) > 2):
        for word in outlier_words:
            if (word in outlier_words): return(word)
            else: return(x)
    else:
        return(x)

tags_cleaned = list(map(lambda x: convert_outliers(x),df_stocks["Tag"]))

df_stocks["Tag"] = tags_cleaned

In [92]:
df_grouped_int = df_stocks.groupby(df_stocks["ticker"]).mean().round().astype(int)
df_grouped_int.reset_index(level=0, inplace=True)
df_grouped_char = df_stocks.groupby(['ticker']).agg({'Tag': ', '.join})
df_grouped_char.reset_index(level=0, inplace=True)
df_ = df_grouped_int.merge(df_grouped_char[["ticker","Tag"]], on = "ticker", how = 'left')

In [57]:
tickers = np.unique(df["ticker"])
maindata = yf.download(list(tickers),start = "2010-01-01")["Adj Close"]

[*********************100%***********************]  182 of 182 completed


In [125]:
stocks = []

for column in maindata:
    if(maindata[column].isna().sum()<round(len(maindata)/2)):
        if(column[-2:] != "TO"):
            stocks.append(column)

In [126]:
data = maindata[stocks].dropna().pct_change().dropna()
df = df_[df_["ticker"].isin(stocks)]
df = df.reset_index()
stocks = data.columns

Model

In [127]:
def run_model(data,minrisk_return):
    # define variables
    stock_volatility = data.std()
    stock_return = data.mean()
    sigma = data.cov()
    stocks = data.columns
    
    m = gp.Model('portfolio')
    n = len(df)

    # Upper bound set to 0.1 to ensure a minimum of 10 stocks, to ensure diversification.
    weights = pd.Series(m.addVars(stocks, vtype = "S", lb = 0.001, ub = 0.1, name = "weights"), index=stocks)

    portfolio_risk = sigma.dot(weights).dot(weights)
    portfolio_return = stock_return.dot(weights)
    m.setObjective(portfolio_risk, GRB.MINIMIZE)
    
    m.addConstr(weights.sum() == 1, 'budget')
    # Ensure a sizable stake in value stock for stability
    m.addConstr(gp.quicksum(weights[i] * df["is.Value"][i] for i in range(n)) >= 0.3, "Value")
    # Given the current situation, our group would like to invest substaintiably into Tech, Alterntive Energy and Pharmaceutical    
    checklist_favor = [int(("Tech" in df.Tag[i]) or ("Alternative" in df.Tag[i]) or ("Pharmaceutical" in df.Tag[i])) for i in range(len(df.Tag))]
    m.addConstr(gp.quicksum(weights[i] * checklist_favor[i] for i in range(n)) >= 0.3, "Situational")

    
    m.addConstr(portfolio_return >= minrisk_return, 'target')

    m.setParam('OutputFlag', 0) # don't print the whole Chunk out
    
    try: 
        m.optimize()
        sharpe = portfolio_risk.getValue()/portfolio_return.getValue()
        result = sharpe
        weights_result = []
        for weight in weights:
            if weight.x > 0:
                weights_result.append([weight.varname[8:-1],weight.x])
                
    except: 
        result = 0
        weights_result = 0
    m.reset()
    return([result,weights_result])

In [128]:
def sim(data):
    data_ = data.iloc[np.random.randint(len(data), size=2520)]
    stock_return = data_.mean()
    ret = np.linspace(0,stock_return.max(), 500)
    
    sharpes = []
    weights = []
    
    result = 1
    i=0
    
    while (result > 0):
        val = run_model(data_,ret[i])
        i = i+1
        result = val[0]
        sharpes.append(val[0])
        weights.append(val[1])
    
    
    return([
        max(sharpes),
        weights[sharpes.index(max(sharpes))]
    ])

In [130]:
# %timeit sim(data) returns 40.2 s ± 19.4 s per loop , q per min, 20 min should be able to run 20 different tries

40.2 s ± 19.4 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [131]:
data_test = data.copy()
data_test = data_test["2020-01-01":]

In [132]:
top = np.sum(data_test,axis=1).sort_values(ascending = False).index[0:28]
bot = np.sum(data_test,axis=1).sort_values().index[0:28]

data_top = data_test[data_test.index.isin(top)].mean()
data_bot = data_test[data_test.index.isin(bot)].mean()
data_average = data_test[~data_test.index.isin((list(top)+list(bot)))].mean()


In [141]:
score = []
weights_ = []
for i in tqdm(range(20)):
    best_sharpe, weights = sim(data)
    total_ret = 0
    for ticker, weight in weights:
        # Given the uncertainty of the current climate, Our group decide to apply the naive method, simply giving an equal weightage to all three possible possibilities 
        total_ret = 1/3*(total_ret + data_top[ticker] * weight) + 1/3*(total_ret + data_bot[ticker] * weight) + 1/3*(total_ret + data_average[ticker] * weight)
    score.append(total_ret)
    weights_.append(weights)


100%|██████████| 20/20 [10:48<00:00, 32.42s/it]


In [144]:
weights_[score.index(max(score))]


[['AMZN', 0.0010579260094780796],
 ['DQ', 0.1],
 ['IMMU', 0.1],
 ['NEE', 0.1],
 ['NLTX', 0.1],
 ['NVAX', 0.1],
 ['NVDA', 0.1],
 ['PLUG', 0.1],
 ['RUP.V', 0.1],
 ['SAM', 0.1],
 ['WLL', 0.09894207399052204]]